In [15]:
import tkinter as tk
from tkinter import filedialog

In [16]:
root = tk.Tk()
root.withdraw()

as_path = filedialog.askopenfilename()
as_path

'C:/Users/lci734/OneDrive - AFRY/Documents/01_poyry/01_projetos/03_usiminas/lingotamento/As Sold/Lingotamento_AS SOLD COMPLETO-2023-06-16.xlsx'

In [17]:
import pandas as pd

In [18]:
from datetime import datetime

def validar_data(prompt):
    while True:
        data_input = input(prompt)
        try:
            data_status = datetime.strptime(data_input, "%d/%m/%Y").date()
            return data_status
        except ValueError:
            print("Data inválida. Por favor, digite uma data no formato DD/MM/YYYY.")

data_status = validar_data("Digite a data de status (DD/MM/YYYY): ")
print("Data validada:", data_status)

Digite a data de status (DD/MM/YYYY): 17/06/2023
Data validada: 2023-06-17


In [19]:
def validar_projeto(prompt):
    while True:
        projeto_input = input(prompt)
        if len(projeto_input) == 13:
            projeto = projeto_input
            return projeto
        else:
            print('Nº do Projeto inválido. Por favor, digite novamente.')
        
projeto = validar_projeto('Digite o Nº do Projeto (1090XXXXX-XXX): ')
print('Nº do Projeto validado:',projeto )

Digite o Nº do Projeto (1090XXXXX-XXX): 109004000-001
Nº do Projeto validado: 109004000-001


In [20]:
tipo = "AS SOLD"

In [21]:
import sqlite3 as sql

In [22]:
con = sql.connect('teste_00.db')
cur = con.cursor()

In [23]:
# criar data frame do as sold com nomes das colunas
df = pd.read_excel(as_path)
df = df[['Unnamed: 1', 'Unnamed: 8', 'Unnamed: 10', 'Unnamed: 11']]
df.columns = ['hours', 'activity', 'category', 'base_cost']
df = df.loc[(df.activity.notnull()) & (df.activity != 'Activity')]

In [24]:
# pegar id_activity
cur.execute("SELECT * FROM activities")
activities = cur.fetchall()
activities = pd.DataFrame(activities)
activities.columns = ['id_activity', 'activity', 'indice', 'unidade', 'id_disciplina']
activities.head()

df = df.merge(activities[['activity', 'id_activity']], 
                           left_on='activity', 
                           right_on='activity',
                           how='left')

df.id_activity = df.id_activity.astype('object')

In [25]:
# inserir categorias novas na tabela categorias
categorias_ineditas = []
for value in list(zip(df.category.values, df.id_activity.values,)):
    cur.execute("SELECT COUNT(*) FROM categorias WHERE categoria = ? AND id_activity = ?", value)
    count = cur.fetchone()[0]
    if count == 0:
        categorias_ineditas.append(value)
        
categorias_ineditas = list(set(categorias_ineditas))
cur.executemany("INSERT INTO categorias (categoria, id_activity) VALUES (?, ?)", categorias_ineditas) 

In [26]:
cur.execute("SELECT * FROM categorias")
categorias = pd.DataFrame(cur.fetchall())
categorias.columns = ['id_categoria', 'categoria', 'id_activity']

df = df.merge(categorias[['categoria', 'id_activity', 'id_categoria']], 
                           left_on=['category','id_activity'], 
                           right_on=['categoria','id_activity'],
                           how='left')

df.id_categoria = df.id_categoria.astype('object')

In [27]:
# inserir projeto na tabela projetos, caso não exista
insert_query = "INSERT OR IGNORE INTO projetos (n_projeto) VALUES (?)"
cur.execute(insert_query, (projeto,))

query = "SELECT * FROM projetos WHERE n_projeto = ?"
cur.execute(query, (projeto,))
id_projeto = cur.fetchall()[0][1]

# inserir registro na tabela registros, caso não exista
check_query = "SELECT COUNT(*) FROM registros WHERE id_projeto = ? AND data_status = ? AND tipo = ?"
cur.execute(check_query, (id_projeto, data_status, tipo,))
unico = cur.fetchall()[0][0]

if unico == 0:
    insert_query = "INSERT OR IGNORE INTO registros (id_projeto, data_status, tipo) VALUES (?, ?, ?)"
    cur.execute(insert_query, (id_projeto, data_status, tipo,))
    print('Registro inserido')
else:
    print('Registro já existente.')

# inserir custo na tabela custos, caso não existam
query = "SELECT * FROM registros WHERE id_projeto = ? AND data_status = ? AND tipo = ?"
cur.execute(query, (id_projeto, data_status, tipo,))
id_registro = cur.fetchall()[0][0]

check_query = "SELECT COUNT(*) FROM custos WHERE id_registro = ?"
cur.execute(check_query, (id_registro,))
unico = cur.fetchall()[0][0]

if unico == 0:  
    id_categoria = df.id_categoria.values  
    base_cost = df.base_cost.values  
    quantidade = df.hours.values
    id_registro = [id_registro] * len(id_categoria)   
    
    insert_query = """INSERT INTO custos 
    (
    id_categoria,
    base_cost,
    quantidade,
    id_registro
    )
    VALUES
    (?, ?, ?, ?)"""
    cur.executemany(insert_query, list(zip(id_categoria, base_cost, quantidade, id_registro,)))
    print('AS SOLD inserido')
else:
    print('AS SOLD já existente.')

Registro inserido
AS SOLD inserido


In [28]:
con.commit()
con.close()